rate limit 
    - If there is a live match, once every hour
    - If there is NO live match, once per day only
so i need to save reponse locally 
https://rapidapi.com/Creativesdev/api/free-api-live-football-data/playground/apiendpoint_20d92b01-b539-43d8-a8df-05c5fad5cd71

In [44]:
#import libs 
import os 
import pandas as pd 
import json
from dotenv import load_dotenv
import requests
import psycopg2
from psycopg2 import OperationalError, Error

In [33]:
#load API key, host and parameters  into py file  
# TO LOAD ENVIRONMENT VARS FROM .env file INTO THE SESSION
load_dotenv()

api_key = os.getenv("API_KEY")
api_host = os.getenv("API_HOST")
league_id = 47
#season = 2020

In [ ]:
#create the API  request with get request 
#CONSTRUCT THE API REQUEST 
url = 'https://free-api-live-football-data.p.rapidapi.com/football-get-standing-all' 
headers = {
    	"x-rapidapi-host" :api_host,
	    "x-rapidapi-key": api_key
}
query_string = {
    "leagueid": league_id
   
}

#SEND THE API REQUEST 
response = requests.get(url,headers=headers, params=query_string)
payload = response.json()
print(payload)

file_name = f"league_{league_id}_standings.json"

#save JSON to a local file in Python
with open(file_name, "w",encoding="utf-8") as f:
    json.dump(payload, f, indent=2,ensure_ascii=False)

file_name = f"league_{league_id}_standings.json"


In [26]:
with open("league_47_standings.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [27]:
data.keys()
#data.values()

dict_keys(['status', 'response'])

In [ ]:
data

In [28]:
#parsing the JSON to isloate data needed 
#parse top level
status = data['status']
standings = data['response']['standing']
teams_count = len(standings)
print(status, teams_count,standings)

success 20 [{'name': 'Arsenal', 'shortName': 'Arsenal', 'id': 9825, 'pageUrl': '/teams/9825/overview/arsenal', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 15, 'draws': 3, 'losses': 2, 'scoresStr': '40-14', 'goalConDiff': 26, 'pts': 48, 'idx': 1, 'qualColor': '#2AD572'}, {'name': 'Manchester City', 'shortName': 'Man City', 'id': 8456, 'pageUrl': '/teams/8456/overview/manchester-city', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '44-18', 'goalConDiff': 26, 'pts': 42, 'idx': 2, 'qualColor': '#2AD572'}, {'name': 'Aston Villa', 'shortName': 'Aston Villa', 'id': 10252, 'pageUrl': '/teams/10252/overview/aston-villa', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '33-24', 'goalConDiff': 9, 'pts': 42, 'idx': 3, 'qualColor': '#2AD572'}, {'name': 'Liverpool', 'shortName': 'Liverpool', 'id': 8650, 'pageUrl': '/teams/8650/overview/liverpool', 'deduction': None, 'ongoing': None

In [29]:
#parse nested json
teams = [team.get('name', 'not found') for team in standings ]
teams

['Arsenal',
 'Manchester City',
 'Aston Villa',
 'Liverpool',
 'Chelsea',
 'Manchester United',
 'Brentford',
 'Sunderland',
 'Newcastle United',
 'Brighton & Hove Albion',
 'Fulham',
 'Everton',
 'Tottenham Hotspur',
 'Crystal Palace',
 'AFC Bournemouth',
 'Leeds United',
 'Nottingham Forest',
 'West Ham United',
 'Burnley',
 'Wolverhampton Wanderers']

In [30]:
#top team with more than 40 pts
top_5_teams = [ team.get('name', 'not found') for team in standings if team['pts']> 40]
top_5_teams

['Arsenal', 'Manchester City', 'Aston Villa']

In [31]:
for team in standings[:2]:
    print("----------------------------")
    print("team_rank:", team['idx'])
    print("team:", team['name'])
    print("pts:", team['pts'])
    print('played:', team['played'])
    print('wins:', team['wins'])
    print('draws:', team['draws'])
    print('losses:', team['losses'])
    scores =  team['scoresStr'].split("-")
    print('goals_for:', int(scores[0]))
    print('goals_against:', int(scores[1]))
    print('goalConDiff:', team['goalConDiff'])
       

----------------------------
team_rank: 1
team: Arsenal
pts: 48
played: 20
wins: 15
draws: 3
losses: 2
goals_for: 40
goals_against: 14
goalConDiff: 26
----------------------------
team_rank: 2
team: Manchester City
pts: 42
played: 20
wins: 13
draws: 3
losses: 4
goals_for: 44
goals_against: 18
goalConDiff: 26


In [34]:
#loop through each team and build a list of reacords
records = []
for team in standings:
   
    scores =  team['scoresStr'].split("-")

    record = (
            team['idx'],
            league_id,
            team['name'],
            team['pts'],
            team['played'],
            team['wins'],
            team['draws'],
            team['losses'],
            int(scores[0]),      # goals_for
            int(scores[1]),      # goals_against
            team['goalConDiff']
        )
    records.append(record)   

In [35]:
columns = ['rank','league_id',
    'name', 'pts', 'played', 'wins', 'draws', 
    'losses', 'goals_for', 'goals_against', 'goals_diff'
]


In [36]:
#create a dataframe of reocrds tuple and list of column names 
df = pd.DataFrame(records, columns=columns)
df.head()

,rank,league_id,name,pts,played,wins,draws,losses,goals_for,goals_against,goals_diff
0,1,47,Arsenal,48,20,15,3,2,40,14,26
1,2,47,Manchester City,42,20,13,3,4,44,18,26
2,3,47,Aston Villa,42,20,13,3,4,33,24,9
3,4,47,Liverpool,34,20,10,4,6,32,28,4
4,5,47,Chelsea,31,20,8,7,5,33,22,11


In [ ]:
df.info()

In [37]:
#load DB creds and connect to Database 
postgresql_host = os.getenv("POSTGRESQL_HOST")
postgresql_port =  os.getenv("POSTGRESQL_PORT")
postgresql_db =  os.getenv("POSTGRESQL_DB")
postgresql_user =  os.getenv("POSTGRESQL_USER")
postgresql_passwd =  os.getenv("POSTGRESQL_PASSWD")

In [63]:
  #flag 
connection = None
try:

    connection = psycopg2.connect(
        host=postgresql_host, 
        port=postgresql_port,
        dbname=postgresql_db,
        user=postgresql_user,
        password=postgresql_passwd

    )

    print('DB connection successfully')
    

except OperationalError as e:
    print(f"{e}, connection failed, connection issues/errors ")

finally:
    if connection:
        cursor = connection.cursor()
        print("Cursor is initiated",cursor)


DB connection successfully
Cursor is initiated <cursor object at 0x000001DE4886E0A0; closed: 0>


In [64]:
#check if table exits 
table_name = "standings"


cursor.execute("""
    SELECT EXISTS (
        SELECT 1
        FROM information_schema.tables
        WHERE table_schema = 'public'
        AND table_name = %s
    );
""", (table_name,))

print('milestone check')

table_exist = cursor.fetchone()[0]

print('milestone check2')

if table_exist:
    print(f'Done, table is exist:{table_name}')
else:
    raise SystemExit(f'Failed, table is not exists')


milestone check
milestone check2
Done, table is exist:standings


In [50]:
records_list = [tuple(row) for row in df.to_numpy()]
records_list

[(1, 47, 'Arsenal', 48, 20, 15, 3, 2, 40, 14, 26),
 (2, 47, 'Manchester City', 42, 20, 13, 3, 4, 44, 18, 26),
 (3, 47, 'Aston Villa', 42, 20, 13, 3, 4, 33, 24, 9),
 (4, 47, 'Liverpool', 34, 20, 10, 4, 6, 32, 28, 4),
 (5, 47, 'Chelsea', 31, 20, 8, 7, 5, 33, 22, 11),
 (6, 47, 'Manchester United', 31, 20, 8, 7, 5, 34, 30, 4),
 (7, 47, 'Brentford', 30, 20, 9, 3, 8, 32, 28, 4),
 (8, 47, 'Sunderland', 30, 20, 7, 9, 4, 21, 19, 2),
 (9, 47, 'Newcastle United', 29, 20, 8, 5, 7, 28, 24, 4),
 (10, 47, 'Brighton & Hove Albion', 28, 20, 7, 7, 6, 30, 27, 3),
 (11, 47, 'Fulham', 28, 20, 8, 4, 8, 28, 29, -1),
 (12, 47, 'Everton', 28, 20, 8, 4, 8, 22, 24, -2),
 (13, 47, 'Tottenham Hotspur', 27, 20, 7, 6, 7, 28, 24, 4),
 (14, 47, 'Crystal Palace', 27, 20, 7, 6, 7, 22, 23, -1),
 (15, 47, 'AFC Bournemouth', 23, 20, 5, 8, 7, 31, 38, -7),
 (16, 47, 'Leeds United', 22, 20, 5, 7, 8, 26, 33, -7),
 (17, 47, 'Nottingham Forest', 18, 20, 5, 3, 12, 19, 33, -14),
 (18, 47, 'West Ham United', 14, 20, 3, 5, 12, 21, 4

In [51]:
upsert_sql = """
    INSERT INTO standings (
    rank, league_id, name, pts, played, wins, draws, losses,
    goals_for, goals_against, goals_diff
)
VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
ON CONFLICT (name)  -- name is primary key
DO UPDATE SET
    rank = EXCLUDED.rank,
    league_id = EXCLUDED.league_id,
    pts = EXCLUDED.pts,
    played = EXCLUDED.played,
    wins = EXCLUDED.wins,
    draws = EXCLUDED.draws,
    losses = EXCLUDED.losses,
    goals_for = EXCLUDED.goals_for,
    goals_against = EXCLUDED.goals_against,
    goals_diff = EXCLUDED.goals_diff;


"""

In [ ]:
try:
    cursor.executemany(upsert_sql, records_list)
    connection.commit()
    print("data inserted/updated successfully....")
except OperationalError as e:
    print(f"{e}, connection failed, connection/operational issues  ")
    connection.rollback()
except Error as e:
    print(f"{e}, connection failed, database/other issues ")
    connection.rollback()
finally:
    conn_status = connection.status
    if conn_status == 1:
        cursor.close()
        connection.close()
        print("all connections are closed cursor and database conn...")

data inserted/updated successfully....
all connections are closed cursor and database conn...
